In [1]:
import sys
sys.path.append('../pygtool3/')
import pygtool
import pygtool_core
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import pandas as pd

# read 2D and 3D data 

In [2]:
bc=pygtool.read3d('./sampledata/mc_bc.clim',count=4)
ps=pygtool.read2d('./sampledata/ps.clim',count=4)
t=pygtool.read3d('./sampledata/T.clim',count=4)

In [ ]:
print('get model values')
print(bc.getarr().shape)
print(ps.getarr(timestep=1).shape)
print(t.getarr(cyclic=True).shape)

In [ ]:
print('get_Datetimeindex')
print(bc.getdatetimeindex())
for i in range(4):
    print(bc.getdate(timestep=i))

In [ ]:
print('get all data into array')
print('3D',bc.getarrays(cyclic=True).shape)
print('2D',ps.getarrays().shape)

# read model coordinate
## horizontal

In [ ]:
print('read t42')
grid=pygtool.readgrid()
print('constructor')
print(grid)
lon,lat=grid.getlonlat()
lonm,latm=grid.getlonlat(cyclic=True)
print(lon.shape,lat.shape,lonm.shape,latm.shape)
xx,yy=grid.getmesh()
print(xx.shape,yy.shape)
print('area caliculation')
area=grid.getarea()
print(area.sum())

print('\nread t85')
grid2=pygtool.readgrid(lonfile='GTAXLOC.GLON256',latfile='GTAXLOC.GGLA128',x=256,y=128)
print(grid2)
lon2,lat2=grid2.getlonlat()
lonm2,latm2=grid2.getlonlat(cyclic=True)
print(lon2.shape,lat2.shape,lonm2.shape,latm2.shape)
xx2,yy2=grid2.getmesh()
print(xx2.shape,yy2.shape)
print('area caliculation')
area2=grid2.getarea()
print(area2.sum())

## vertical(sigma)

In [ ]:
print('vertical coordinate layer 36')
sigma=pygtool.readsigma()
print(sigma)
print('get pressure by ps(Gtool2d)')
print(sigma.get_pressure(ps,timestep=1)[0,0,0])
print('get pressure by ps(numpy.ndarray)')
print(sigma.get_pressure(ps.getarr(timestep=1))[0,0,0])

print('\n layer57')
sigma2=pygtool.readsigma(z=57,gtaxfile='GTAXLOC.CETA57')
print('get pressure by ps(Gtool2d)')
print(sigma2.get_pressure(ps,timestep=1)[0,0,0])
print('get pressure by ps(numpy.ndarray)')
print(sigma2.get_pressure(ps.getarr(timestep=1))[0,0,0])

## vertical(P)

In [ ]:
print('vertical coodinate(P-grid)')
P=pygtool.readpressure()
print(P)
print('get pressure')
print(P.get_pressure()[:,0,0])

In [ ]:
print('vertical coodinate(P-grid,boundary) ')
PM=pygtool.readpressure(z=36,gtaxfile='GTAXLOC.AR5PL35.M')
print(PM)
print('get dp')
print(PM.get_dp()[:,0,0])

# gtool3 => xarray

In [ ]:
ds=bc.to_xarray(lon=lon,lat=lat,sigma=sigma.ss,creator='onishi')
ds

In [ ]:
fig=plt.figure(figsize=(10,5),facecolor='w')
ax=fig.add_subplot(1,1,1,projection=ccrs.PlateCarree(central_longitude=180))
pygtool.ckit.set_geogrid(ax)
ds['CONBT'].isel(time=2,sigma=0).plot.contourf(ax=ax,transform=ccrs.PlateCarree())


In [ ]:
bc.getdatetimeindex()

In [ ]:
P=pygtool.readpressure()
P

In [ ]:
P.get_pressure().shape

In [ ]:
PM=pygtool.readpressure(z=36,gtaxfile='GTAXLOC.AR5PL35.M')

In [ ]:
PM.get_dp().shape

# python =>gtool

In [ ]:
a=0

In [3]:
gtool=pygtool_core.Gtool()
gtool.set_sampleheader(sample=bc.getheader())

0

In [4]:
datetime=bc.getdatetimeindex()
datetime

DatetimeIndex(['2014-12-01 12:00:00', '2014-03-01 12:00:00',
               '2014-06-01 12:00:00', '2014-09-01 12:00:00'],
              dtype='datetime64[ns]', freq=None)

In [5]:
datalist=[]
for i in range(4):
    datalist.append(bc.getarr(i))

In [6]:
datalist[0].shape

(36, 64, 128)

In [7]:
gtool.to_gtool(datalist=datalist,datetimeindex=datetime)

In [ ]:
for dt in datetime:
    print(dt)

In [ ]:
dt.strftime('%Y%m%d 000000')

In [ ]:
gtool.data['header'].shape